#### https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf

In [1]:
from datasets import load_dataset

/home/anishdabhane/PCSB/RoBERTa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# dataset cha path change kr karshil
dataset=load_dataset('csv',data_files={'train':'final.csv'})

Generating train split: 0 examples [00:00, ? examples/s]/home/anishdabhane/PCSB/RoBERTa/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 6615 examples [00:00, 149744.02 examples/s]


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'tweet'],
        num_rows: 6615
    })
})


In [11]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [12]:
print(tokenizer("hello"))

{'input_ids': [101, 7592, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [17]:
def tokenize_data(example):
    return tokenizer(example['tweet'],padding='max_length')

In [18]:
dataset=dataset.map(tokenize_data,batched=True)

Map: 100%|██████████| 6615/6615 [00:03<00:00, 1863.02 examples/s]


In [22]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6615
    })
})


### Training

In [24]:
from transformers import TrainingArguments

training_args=TrainingArguments("test_trainer",num_train_epochs=3)

In [26]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
#splitting the dataset 
#baad mein karunga
train_dataset=dataset['train'].shuffle(seed=10).select(range(5000))
eval_dataset=dataset['train'].shuffle(seed=10).select(range(5000,6615))


In [37]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['sentiment', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})
Dataset({
    features: ['sentiment', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1615
})


In [40]:
from transformers import Trainer

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [41]:
trainer.train()

  0%|          | 0/1875 [00:00<?, ?it/s]

KeyboardInterrupt: 